In [2]:
# !pip install tensorflow
# !pip install scikit-learn
!pip install optuna
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install timm
# # or for huggingface transformers if you'd like to use that:
!pip install transformers
!pip install matplotlib opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.2 MB/s eta 0:00:00a 0:00:01


In [2]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("sivm205/soybean-diseased-leaf-dataset")

# print("Path to dataset files:", path)

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Import PyTorch Geometric modules
from torch_geometric.nn import GCNConv, global_mean_pool

import optuna


In [4]:
import tensorflow as tf

print(f"GPU available: {tf.test.is_gpu_available()}")
print(f"CUDA devices: {tf.config.list_physical_devices('GPU')}")
print(f"cuDNN version: {tf.test.gpu_device_name()}")

GPU available: True
CUDA devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
cuDNN version: /device:GPU:0


In [5]:
import os

# Dataset Directory
dataset_dir = "/kaggle/input/soybean-diseased-leaf-dataset"
os.listdir(dataset_dir)

['ferrugen',
 'Yellow Mosaic',
 'brown_spot',
 'powdery_mildew',
 'septoria',
 'Southern blight',
 'Mossaic Virus',
 'Sudden Death Syndrone',
 'bacterial_blight',
 'crestamento']

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, ResNet50, VGG16, VGG19, Xception, DenseNet121, DenseNet169
from tensorflow.keras.applications import DenseNet201, InceptionV3, InceptionResNetV2, NASNetLarge, NASNetMobile, ResNet101, ResNet152, ResNet50V2
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import networkx as nx
import scipy.sparse as sp
import os
import cv2
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import optuna
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Dataset Directory
dataset_dir = "/kaggle/input/soybean-diseased-leaf-dataset"

# Load dataset images and labels
def load_dataset(dataset_dir, img_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    class_dict = {class_name: idx for idx, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_path = os.path.join(dataset_dir, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize image
            images.append(img)
            labels.append(class_dict[class_name])

    return np.array(images), np.array(labels)

# Load actual dataset
images, labels = load_dataset(dataset_dir)

# Data Preprocessing & Augmentation
def preprocess_data(X, y):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        rescale=1./255
    )
    return datagen.flow(X, y, batch_size=32)


# Define CNN models to benchmark (add more as needed)
CNN_MODELS = {
    'MobileNetV2': MobileNetV2,
    'EfficientNetB0': EfficientNetB0,
    'ResNet50': ResNet50,
    'VGG16': VGG16,
    'VGG19': VGG19,
    'Xception': Xception,
    'DenseNet121': DenseNet121,
    'DenseNet169': DenseNet169,
    'DenseNet201': DenseNet201,
    # 'InceptionV3': InceptionV3,
    # 'InceptionResNetV2': InceptionResNetV2,
    # 'NASNetLarge': NASNetLarge,
    # 'NASNetMobile': NASNetMobile,
    # 'ResNet101': ResNet101,
    # 'ResNet152': ResNet152,
    # 'ResNet50V2': ResNet50V2
}

# Define GNN configurations (example layer sizes)
GNN_MODELS = {
    'GCN': [64, 32],
    'GAT': [128, 64],
    'GraphSAGE': [256, 128]
}

def create_cnn(input_shape, cnn_name):
    base_model_class = CNN_MODELS[cnn_name]
    base_model = base_model_class(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(128, activation='relu')(x)
    return tf.keras.Model(inputs, outputs)

def extract_features(images, cnn_name):
    cnn_model = create_cnn((224, 224, 3), cnn_name)
    batch_size = 16  # Reduce batch size to prevent memory overflow
    return cnn_model.predict(images, batch_size=batch_size)

def create_graph(image_features, threshold=0.8):
    similarity_matrix = cosine_similarity(image_features)
    adj_matrix = (similarity_matrix > threshold).astype(int)
    return sp.coo_matrix(adj_matrix)

def create_gnn(input_dim, gnn_layers):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = inputs
    for units in gnn_layers:
        x = layers.Dense(units, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

def build_sequential_model(cnn_name, gnn_layers, input_shape):
    cnn_model = create_cnn(input_shape, cnn_name)
    gnn_input_dim = cnn_model.output_shape[-1]
    gnn_model = create_gnn(gnn_input_dim, gnn_layers)
    return tf.keras.Model(inputs=cnn_model.input, outputs=gnn_model(cnn_model.output))


# Split data into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

results = []

for cnn_name in CNN_MODELS.keys():
    print(f"Testing CNN: {cnn_name}")
    # Extract features and create graph using training data
    train_features = extract_features(train_data, cnn_name)
    adj_matrix = create_graph(train_features)  # Not used in current GNN setup

    for gnn_name, gnn_layers in GNN_MODELS.items():
        print(f"  Testing GNN: {gnn_name}")
        # Build and compile the model
        model = build_sequential_model(cnn_name, gnn_layers, (224, 224, 3))
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Train the model
        history = model.fit(
            train_data,
            tf.keras.utils.to_categorical(train_labels, num_classes=10),
            epochs=20,
            batch_size=32,
            validation_data=(
                test_data,
                tf.keras.utils.to_categorical(test_labels, num_classes=10)
            ),
            verbose=0
        )

        # Evaluate on test data
        predictions = model.predict(test_data)
        predicted_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(tf.keras.utils.to_categorical(test_labels, num_classes=10), axis=1)

        # Calculate metrics
        acc = accuracy_score(true_labels, predicted_labels)
        precision = precision_score(true_labels, predicted_labels, average='weighted')
        recall = recall_score(true_labels, predicted_labels, average='weighted')
        f1 = f1_score(true_labels, predicted_labels, average='weighted')

        # Store results
        results.append({
            'CNN': cnn_name,
            'GNN': gnn_name,
            'Accuracy': acc,
            'Precision': precision,
            'Recall': recall,
            'F1': f1
        })

# Print benchmarking results
print("\nBenchmark Results:")
print("{:<15} {:<15} {:<8} {:<9} {:<6} {}".format(
    "CNN Model", "GNN Model", "Accuracy", "Precision", "Recall", "F1 Score"))
print("-" * 65)
for res in results:
    print("{:<15} {:<15} {:.4f}   {:.4f}    {:.4f}  {:.4f}".format(
        res['CNN'], res['GNN'], res['Accuracy'], res['Precision'], res['Recall'], res['F1']))

Testing CNN: MobileNetV2
35/35 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 664ms/step
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 659ms/step
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 644ms/step
Testing CNN: EfficientNetB0
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step
  Testing GNN: GAT


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step
  Testing GNN: GraphSAGE


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step
Testing CNN: ResNet50


...

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 256ms/step
  Testing GNN: GraphSAGE


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 260ms/step
Testing CNN: Xception


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 635ms/step
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 642ms/step
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 644ms/step
Testing CNN: DenseNet121
35/35 ━━━━━━━━━━━━━━━━━━━━ 14s 41ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step
Testing CNN: DenseNet169
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 51ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step
Testing CNN: DenseNet201
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 66ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step 
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step 
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step 

Benchmark Results:
CNN Model       GNN Model       Accu

In [3]:
import gc
gc. collect()

0

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, ResNet50, VGG16, VGG19, Xception, DenseNet121, DenseNet169
from tensorflow.keras.applications import DenseNet201, InceptionV3, InceptionResNetV2, NASNetLarge, NASNetMobile, ResNet101, ResNet152, ResNet50V2
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import networkx as nx
import scipy.sparse as sp
import os
import cv2
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import optuna
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Dataset Directory
dataset_dir = "/kaggle/input/soybean-diseased-leaf-dataset"

# Load dataset images and labels
def load_dataset(dataset_dir, img_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    class_dict = {class_name: idx for idx, class_name in enumerate(class_names)}

    for class_name in class_names:
        class_path = os.path.join(dataset_dir, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize image
            images.append(img)
            labels.append(class_dict[class_name])

    return np.array(images), np.array(labels)

# Load actual dataset
images, labels = load_dataset(dataset_dir)

# Data Preprocessing & Augmentation
def preprocess_data(X, y):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        rescale=1./255
    )
    return datagen.flow(X, y, batch_size=32)


# Define CNN models to benchmark (add more as needed)
CNN_MODELS = {
    # 'MobileNetV2': MobileNetV2,
    # 'EfficientNetB0': EfficientNetB0,
    # 'ResNet50': ResNet50,
    # 'VGG16': VGG16,
    # 'VGG19': VGG19,
    # 'Xception': Xception,
    # 'DenseNet121': DenseNet121,
    # 'DenseNet169': DenseNet169,
    # 'DenseNet201': DenseNet201,
    'InceptionV3': InceptionV3,
    'NASNetLarge': NASNetLarge,
    'ResNet101': ResNet101,
    'ResNet152': ResNet152,
    'ResNet50V2': ResNet50V2
}

# Define GNN configurations (example layer sizes)
GNN_MODELS = {
    'GCN': [64, 32],
    'GAT': [128, 64],
    'GraphSAGE': [256, 128]
}

def create_cnn(input_shape, cnn_name):
    base_model_class = CNN_MODELS[cnn_name]
    base_model = base_model_class(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(128, activation='relu')(x)
    return tf.keras.Model(inputs, outputs)

def extract_features(images, cnn_name):
    cnn_model = create_cnn((224, 224, 3), cnn_name)
    batch_size = 16  # Reduce batch size to prevent memory overflow
    return cnn_model.predict(images, batch_size=batch_size)

def create_graph(image_features, threshold=0.8):
    similarity_matrix = cosine_similarity(image_features)
    adj_matrix = (similarity_matrix > threshold).astype(int)
    return sp.coo_matrix(adj_matrix)

def create_gnn(input_dim, gnn_layers):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = inputs
    for units in gnn_layers:
        x = layers.Dense(units, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

def build_sequential_model(cnn_name, gnn_layers, input_shape):
    cnn_model = create_cnn(input_shape, cnn_name)
    gnn_input_dim = cnn_model.output_shape[-1]
    gnn_model = create_gnn(gnn_input_dim, gnn_layers)
    return tf.keras.Model(inputs=cnn_model.input, outputs=gnn_model(cnn_model.output))


# Split data into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

results = []

for cnn_name in CNN_MODELS.keys():
    print(f"Testing CNN: {cnn_name}")
    # Extract features and create graph using training data
    train_features = extract_features(train_data, cnn_name)
    adj_matrix = create_graph(train_features)  # Not used in current GNN setup

    for gnn_name, gnn_layers in GNN_MODELS.items():
        print(f"  Testing GNN: {gnn_name}")
        # Build and compile the model
        model = build_sequential_model(cnn_name, gnn_layers, (224, 224, 3))
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Train the model
        history = model.fit(
            train_data,
            tf.keras.utils.to_categorical(train_labels, num_classes=10),
            epochs=20,
            batch_size=32,
            validation_data=(
                test_data,
                tf.keras.utils.to_categorical(test_labels, num_classes=10)
            ),
            verbose=0
        )

        # Evaluate on test data
        predictions = model.predict(test_data)
        predicted_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(tf.keras.utils.to_categorical(test_labels, num_classes=10), axis=1)

        # Calculate metrics
        acc = accuracy_score(true_labels, predicted_labels)
        precision = precision_score(true_labels, predicted_labels, average='weighted')
        recall = recall_score(true_labels, predicted_labels, average='weighted')
        f1 = f1_score(true_labels, predicted_labels, average='weighted')

        # Store results
        results.append({
            'CNN': cnn_name,
            'GNN': gnn_name,
            'Accuracy': acc,
            'Precision': precision,
            'Recall': recall,
            'F1': f1
        })

# Print benchmarking results
print("\nBenchmark Results:")
print("{:<15} {:<15} {:<8} {:<9} {:<6} {}".format(
    "CNN Model", "GNN Model", "Accuracy", "Precision", "Recall", "F1 Score"))
print("-" * 65)
for res in results:
    print("{:<15} {:<15} {:.4f}   {:.4f}    {:.4f}  {:.4f}".format(
        res['CNN'], res['GNN'], res['Accuracy'], res['Precision'], res['Recall'], res['F1']))

Testing CNN: InceptionV3
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step 
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step 
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step
Testing CNN: NASNetLarge
343610240/343610240 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 28s 195ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step 
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step 
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step 
Testing CNN: ResNet101
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step 
  Testing GNN: GAT


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step 
  Testing GNN: GraphSAGE


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step 
Testing CNN: ResNet152


...

5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step 
  Testing GNN: GraphSAGE


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step 
Testing CNN: ResNet50V2


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step
  Testing GNN: GCN
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 753ms/step
  Testing GNN: GAT
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 761ms/step
  Testing GNN: GraphSAGE
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 836ms/step

Benchmark Results:
CNN Model       GNN Model       Accuracy Precision Recall F1 Score
-----------------------------------------------------------------
InceptionV3     GCN             0.9645   0.9670    0.9645  0.9605
InceptionV3     GAT             0.9504   0.9555    0.9504  0.9421
InceptionV3     GraphSAGE       0.9716   0.9746    0.9716  0.9706
NASNetLarge     GCN             0.9291   0.9359    0.9291  0.9295
NASNetLarge     GAT             0.9149   0.9141    0.9149  0.9133
NASNetLarge     GraphSAGE       0.9149   0.9157    0.9149  0.9127
ResNet101       GCN             0.6241   0.6652    0.6241  0.5618
ResNet101       GAT             0.7730   0.7459    0.7730  0.7290
ResNet101       GraphSAGE       0.8085   0.7969

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import (
    MobileNetV2, EfficientNetB0, ResNet50, VGG16, VGG19, Xception, 
    DenseNet121, DenseNet169, DenseNet201, InceptionV3, 
    InceptionResNetV2, NASNetLarge, NASNetMobile, 
    ResNet101, ResNet152, ResNet50V2
)

# Define CNN models
CNN_MODELS = {
    'MobileNetV2': MobileNetV2,
    'EfficientNetB0': EfficientNetB0,
    'ResNet50': ResNet50,
    'VGG16': VGG16,
    'VGG19': VGG19,
    'Xception': Xception,
    'DenseNet121': DenseNet121,
    'DenseNet169': DenseNet169,
    'DenseNet201': DenseNet201,
    'InceptionV3': InceptionV3,
    'InceptionResNetV2': InceptionResNetV2,
    'NASNetLarge': NASNetLarge,
    'NASNetMobile': NASNetMobile,
    'ResNet101': ResNet101,
    'ResNet152': ResNet152,
    'ResNet50V2': ResNet50V2
}

# Define GNN architectures
GNN_MODELS = {
    'GCN': [64, 32],         # Two-layer GCN
    'GAT': [128, 64],        # Two-layer GAT
    'GraphSAGE': [256, 128]  # Two-layer GraphSAGE
}

# Function to create CNN backbone
def create_cnn(input_shape, cnn_name):
    base_model_class = CNN_MODELS[cnn_name]
    base_model = base_model_class(input_shape=input_shape, include_top=False, weights=None)
    return base_model

# Function to create GNN model
def create_gnn(input_dim, gnn_layers):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = inputs
    for units in gnn_layers:
        x = layers.Dense(units, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)  # Assuming 10 output classes
    model = tf.keras.Model(inputs, outputs)
    return model

# Print parameter counts
print("{:<20} {:<15}".format("Model Name", "Parameter Count"))
print("=" * 40)

# Print CNN model parameter counts
for model_name, model_class in CNN_MODELS.items():
    model = create_cnn((224, 224, 3), model_name)
    param_count = model.count_params()
    print("{:<20} {:<15,}".format(model_name, param_count))

# Print GNN model parameter counts
for gnn_name, gnn_layers in GNN_MODELS.items():
    gnn_model = create_gnn(128, gnn_layers)  # Assuming CNN extracts 128 features
    param_count = gnn_model.count_params()
    print("{:<20} {:<15,}".format(gnn_name, param_count))


Model Name           Parameter Count
MobileNetV2          2,257,984      
EfficientNetB0       4,049,571      
ResNet50             23,587,712     
VGG16                14,714,688     
VGG19                20,024,384     
Xception             20,861,480     
DenseNet121          7,037,504      
DenseNet169          12,642,880     
DenseNet201          18,321,984     
InceptionV3          21,802,784     
InceptionResNetV2    54,336,736     
NASNetLarge          84,916,818     
NASNetMobile         4,269,716      
ResNet101            42,658,176     
ResNet152            58,370,944     
ResNet50V2           23,564,800     
GCN                  10,666         
GAT                  25,418         
GraphSAGE            67,210         
